# Tarea 2

* Sofía Cruz T. (**@cutsof**)
* Daniel Martínez U. (**@danmarurr**)

**Envío del PR inicial:** lunes 5 de septiembre

**Aceptación del PR:** lunes 19 de septiembre

**NOTA** El máximo número de colaboradores es 2, cuyos nombres deben ser incluidos con claridad a la hora de hacer el PR, por ejemplo, mandándo copia a los coautores. Si usan ideas de cualquier fuente, favor de incluir la referencia apropiada. (En otras palabras, sí está permitido usar otras fuentes, pero incluyan las referencias.)

---

1) Consideren la función

\begin{equation}
z_{n+1} = z_n^2 + c, 
\end{equation}

con `c = 0.12256484161600012 - 0.74486407104im`. Definan una malla de valores iniciales $z_0$ en el intervalo $[-1,1]\times [-1,1]$ (parte real $\times$ parte imaginaria).

- ¿Cuantos valores finitos distintos (en el intervalo $[-1,1]\times [-1,1]$) se pueden obtener después de un gran número de iteraciones?

- Usando un color distinto para cada uno, grafiquen en el plano $\textrm{Re}(z) - \textrm{Im}(z)$ el conjunto obtenido.

# Problema 1

Para resolver este problema implementaremos tres funciones: `mapeo1` para calcular el mapeo, `crea_malla` para crear la malla en $[-1,1]\times[-1,1]$ y finalmente `conteo1` para contar los valores finitos distintos.

In [19]:
using PyPlot

In [20]:
sqrt(0.12256484161600012 - 0.74486407104im)

0.6623614490371831 - 0.5622791544727911im

In [21]:
f(0.0 + 0*im)

LoadError: LoadError: UndefVarError: f not defined
while loading In[21], in expression starting on line 1

In [22]:
"""
`mapeo1` calcula el término `n` de la función z_{n+1} = z_n^2 + c, donde 
c = 0.12256484161600012 - 0.74486407104im y `z0` es un valor definido por el usuario.
"""
function mapeo1(n::Int64, z0)
    @assert n≥1 "n tiene que ser mayor o igual a 0"
    c = complex(0.12256484161600012,-0.74486407104)
   
    for j in 1:n
        z0 = copy(z0)^2 - c
    end
    return z0
end

"""
`crea_malla` crea una malla en `domRe`x`domIm` usando separaciones Δx = Δ[1] y Δy = Δ[2]. Se regresa una matriz 
con entradas del tipo `Complex{Float64}` con dicha malla.

***Notas:***

- El valor predeterminado para Δ es [0.025, 0.025].

- `domRe` y `domIm` deben ser arreglos de `Float64` con la siguiente información:

`domRe` = [`x_min`, `x_max`], `domIm` = [`y_min`, `y_max`]
"""
function crea_malla(domRe::Array{Float64}, domIm::Array{Float64}, Δ = [0.0025, 0.0025])
    @assert length(domRe) == 2 && length(domIm) == 2 && length(Δ) == 2 "Uno o varios de los argumentos tienen información faltante o sobrante"
    @assert Δ[1] > 0 && Δ[2]> 0 "El valor de separación debe ser positivo"
    @assert Δ[1] < abs(domRe[1] - domRe[2]) && Δ[2] < abs(domIm[1] - domIm[2]) "El valor de separación debe ser menor al tamaño de los intervalos"
    δx = collect(domRe[1]:Δ[1]:domRe[2])
    δy = collect(domIm[1]:Δ[2]:domIm[2])
    n = length(δx)
    m = length(δy)
    malla = zeros(Complex{Float64}, n, m)
    #Llenamos la malla
    for j in 1:m
        for i in 1:n
            malla[i, j] = δx[j] + im*δy[m + 1 - i]
        end
    end
    return malla
end

"""
`complexmap` mapea la función compleja `f` en la malla de números complejos `malla`
"""
function complexmap(f::Function, malla::Array{Complex{Float64},2})
    fz = zeros(malla)
    for j in 1:length(malla)
        fz[j] = f(malla[j])
    end
    return fz
end

"""
`problema1` realiza el mapeo de la función considerada en el problema 1 en el dominio considerado (por defecto), usando
`n` iteraciones.
"""
function problema1(n::Int, domRe = [-1.0,1.0], domIm = [-1.0,1.0], Δ = [0.0025, 0.0025], f = mapeo1)
    @assert n ≥ 0 "n debe ser mayor o igual a 0"
    @assert length(domRe) == 2 && length(domIm) == 2 && length(Δ) == 2 "Uno o varios de los argumentos tienen información faltante o sobrante"
    @assert Δ[1] > 0 && Δ[2]> 0 "El valor de separación debe ser positivo"
    @assert Δ[1] < abs(domRe[1] - domRe[2]) && Δ[2] < abs(domIm[1] - domIm[2]) "El valor de separación debe ser menor al tamaño de los intervalos"
    malla1 = crea_malla(domRe, domIm, Δ)
    f(x) = mapeo1(n, x)
    fm = complexmap(f, malla1)
    
    zm1 = zeros(fm)
    for j in 1:length(fm)
        if abs(fm[j].re) < 1 && abs(fm[j].im) < 1
            zm1[j] = fm[j]
        end
    end
    return zm1
end

problema1 (generic function with 5 methods)

In [23]:
ab = problema1(1000);

In [24]:
imshow(imag(ab),cmap = "Blues", extent =[-1.0, 1.0, -1.0, 1.0],aspect="auto")

xlabel(L"\Re(z)")
ylabel(L"\Im(z)")
colorbar()
title(L"Parte imaginaria de iteración $n=1000$")

LoadError: LoadError: PyError (:PyObject_Call) <type 'exceptions.UnicodeDecodeError'>
UnicodeDecodeError('ascii', 'Parte imaginaria de iteraci\xc3\xb3n $n=1000$', 27, 28, 'ordinal not in range(128)')
  File "/home/sof9409/anaconda/lib/python2.7/site-packages/matplotlib/pyplot.py", line 1349, in title
    l =  gca().set_title(s, *args, **kwargs)
  File "/home/sof9409/anaconda/lib/python2.7/site-packages/matplotlib/axes/_axes.py", line 142, in set_title
    title.set_text(label)
  File "/home/sof9409/anaconda/lib/python2.7/site-packages/matplotlib/text.py", line 1028, in set_text
    self._text = '%s' % (s,)

while loading In[24], in expression starting on line 6

In [11]:
imshow(real(ab),cmap = "Blues", extent =[-1.0, 1.0, -1.0, 1.0],aspect="auto")

xlabel(L"\Re(z)")
ylabel(L"\Im(z)")
colorbar()
title(L"Parte real de iteración $n=1000$")

LoadError: LoadError: PyError (:PyObject_Call) <type 'exceptions.UnicodeDecodeError'>
UnicodeDecodeError('ascii', 'Parte real de iteraci\xc3\xb3n $n=1000$', 21, 22, 'ordinal not in range(128)')
  File "/home/sof9409/anaconda/lib/python2.7/site-packages/matplotlib/pyplot.py", line 1349, in title
    l =  gca().set_title(s, *args, **kwargs)
  File "/home/sof9409/anaconda/lib/python2.7/site-packages/matplotlib/axes/_axes.py", line 142, in set_title
    title.set_text(label)
  File "/home/sof9409/anaconda/lib/python2.7/site-packages/matplotlib/text.py", line 1028, in set_text
    self._text = '%s' % (s,)

while loading In[11], in expression starting on line 6

In [12]:
mod(2.7,1)

0.7000000000000002

In [13]:
length(zm1)

LoadError: LoadError: UndefVarError: zm1 not defined
while loading In[13], in expression starting on line 1

2) Usando la función `compute_roots` vista en clase, o en alguna variación propuesta por ustedes, definan (en Julia) el mapeo

\begin{equation}
x_{n+1} = B(x_{n}) = 2x_n \mod 1.
\end{equation}

- Obtengas los puntos fijos de este mapeo numéricamente. Los puntos fijos del mapeo son aquéllos que se mapean en si mismos, es decir, $x_{n} = B(x_{x_n})$. Este cálculo se puede hacer a mano, lo que debe usarse para comprobar que la implementación numérica funciona bien.

- Pensando en que la estabilidad de un punto fijo es el comportamiento que se observa de los iterados (a priori un número infinito) al considerar pequeñas desviaciones del punto fijo, ¿cuál es la estabilidad de los puntos fijos que se obtiene *numéricamente*, digamos, después de 100 iteraciones? ¿Pueden encontrar una explicación analítica de la observación numérica?

*Hint: Sean imaginos y no se conformen*

In [160]:
workspace()

Definimos un tipo que nos va a mostrar el nuestra semilla x_0 , al lado del valor al que llegó después de la iteración

In [161]:
type Binter{T<:Real} <: Real
    seed :: T
    iter :: T
end

In [162]:
"""
`iteracion` calcula el término `n` de la función x_{n+1} = B(x_{n}) = 2x_n \mod 1, donde 
se comienza co un x_0 como valor definido por el usuario. 
"""

function iteracion(n::Int, x::Binter)
     k = x.seed
    for j in 1:n
        k = mod(2k,1)
    end
    return Binter(x.seed,k)
end

iteracion (generic function with 1 method)

In [168]:
"""
`mapeo2` calcula `iteracion` en un rango. El rango decide el tipo de las componentes de Binter.
"""

function mapeo2(start,intervalo,final)
    y=Binter[iteracion(100, Binter(start,start))]
    for i in start+intervalo:intervalo:final
        push!(y,iteracion(100, Binter(i,i))) 
    end
    return y
end

mapeo2 (generic function with 1 method)

Encontramos que cualquier irracional pega a cero porque Float64 arrastra errores

In [173]:
mapeo2(0.0,π/100,10*π)

1000-element Array{Binter{T<:Real},1}:
 Binter{Float64}(0.0,0.0)                 
 Binter{Float64}(0.031415926535897934,0.0)
 Binter{Float64}(0.06283185307179587,0.0) 
 Binter{Float64}(0.0942477796076938,0.0)  
 Binter{Float64}(0.12566370614359174,0.0) 
 Binter{Float64}(0.15707963267948966,0.0) 
 Binter{Float64}(0.18849555921538758,0.0) 
 Binter{Float64}(0.21991148575128555,0.0) 
 Binter{Float64}(0.25132741228718347,0.0) 
 Binter{Float64}(0.2827433388230814,0.0)  
 Binter{Float64}(0.3141592653589793,0.0)  
 Binter{Float64}(0.34557519189487723,0.0) 
 Binter{Float64}(0.3769911184307752,0.0)  
    ⋮                                     
 Binter{Float64}(31.03893541746716,0.0)   
 Binter{Float64}(31.070351344003058,0.0)  
 Binter{Float64}(31.101767270538957,0.0)  
 Binter{Float64}(31.133183197074853,0.0)  
 Binter{Float64}(31.164599123610753,0.0)  
 Binter{Float64}(31.19601505014665,0.0)   
 Binter{Float64}(31.227430976682548,0.0)  
 Binter{Float64}(31.258846903218444,0.0)  
 Binter{Float64

En cambio si trabajamos con racionales encontramos que podemos observar la periodicidad que todos tienene , excepto aquellos de la forma $\frac{1}{2^n}$

In [180]:
mapeo2(0//1,1//(2^6-1),1//1)

64-element Array{Binter{T<:Real},1}:
 Binter{Rational{Int64}}(0//1,0//1)    
 Binter{Rational{Int64}}(1//63,16//63) 
 Binter{Rational{Int64}}(2//63,32//63) 
 Binter{Rational{Int64}}(1//21,16//21) 
 Binter{Rational{Int64}}(4//63,1//63)  
 Binter{Rational{Int64}}(5//63,17//63) 
 Binter{Rational{Int64}}(2//21,11//21) 
 Binter{Rational{Int64}}(1//9,7//9)    
 Binter{Rational{Int64}}(8//63,2//63)  
 Binter{Rational{Int64}}(1//7,2//7)    
 Binter{Rational{Int64}}(10//63,34//63)
 Binter{Rational{Int64}}(11//63,50//63)
 Binter{Rational{Int64}}(4//21,1//21)  
    ⋮                                  
 Binter{Rational{Int64}}(52//63,13//63)
 Binter{Rational{Int64}}(53//63,29//63)
 Binter{Rational{Int64}}(6//7,5//7)    
 Binter{Rational{Int64}}(55//63,61//63)
 Binter{Rational{Int64}}(8//9,2//9)    
 Binter{Rational{Int64}}(19//21,10//21)
 Binter{Rational{Int64}}(58//63,46//63)
 Binter{Rational{Int64}}(59//63,62//63)
 Binter{Rational{Int64}}(20//21,5//21) 
 Binter{Rational{Int64}}(61//63,31//63)
 Bi

In [179]:
mapeo2(0//1,1//(2^6),1//1)

65-element Array{Binter{T<:Real},1}:
 Binter{Rational{Int64}}(0//1,0//1)  
 Binter{Rational{Int64}}(1//64,0//1) 
 Binter{Rational{Int64}}(1//32,0//1) 
 Binter{Rational{Int64}}(3//64,0//1) 
 Binter{Rational{Int64}}(1//16,0//1) 
 Binter{Rational{Int64}}(5//64,0//1) 
 Binter{Rational{Int64}}(3//32,0//1) 
 Binter{Rational{Int64}}(7//64,0//1) 
 Binter{Rational{Int64}}(1//8,0//1)  
 Binter{Rational{Int64}}(9//64,0//1) 
 Binter{Rational{Int64}}(5//32,0//1) 
 Binter{Rational{Int64}}(11//64,0//1)
 Binter{Rational{Int64}}(3//16,0//1) 
    ⋮                                
 Binter{Rational{Int64}}(53//64,0//1)
 Binter{Rational{Int64}}(27//32,0//1)
 Binter{Rational{Int64}}(55//64,0//1)
 Binter{Rational{Int64}}(7//8,0//1)  
 Binter{Rational{Int64}}(57//64,0//1)
 Binter{Rational{Int64}}(29//32,0//1)
 Binter{Rational{Int64}}(59//64,0//1)
 Binter{Rational{Int64}}(15//16,0//1)
 Binter{Rational{Int64}}(61//64,0//1)
 Binter{Rational{Int64}}(31//32,0//1)
 Binter{Rational{Int64}}(63//64,0//1)
 Binter{Ratio

In [125]:
function pfijos(f, dom = collect(-1:0.025:1))
    it = zeros(dom)
    for j in 1:length(dom)
        it[j] = f(dom[j])
    end
    return it, abs(it - dom)
end

pfijos (generic function with 2 methods)

In [128]:
y = [h(k)for k in 1//123:1//(2^5-1):100//1]

3100-element Array{Rational{Int64},1}:
   40//123 
 1363//3813
 1486//3813
 1609//3813
 1732//3813
 1855//3813
 1978//3813
 2101//3813
 2224//3813
 2347//3813
 2470//3813
 2593//3813
 2716//3813
     ⋮     
 3577//3813
 3700//3813
   10//3813
  133//3813
  256//3813
  379//3813
  502//3813
  625//3813
  748//3813
  871//3813
  994//3813
 1117//3813

In [103]:
mapeo2(100,1.0)

0.0

In [31]:
?mapeo2

search: 

No documentation found.

`mapeo2` is a generic `Function`.

```julia
# 1 method for generic function "mapeo2":
mapeo2(n::Int64, x::Float64) at In[27]:2
```


mapeo2 mapeo1

